<a href="https://colab.research.google.com/github/VintageGold/Outages_Prediction/blob/master/WeatherStation_Criteria_RawAllNOAA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#This notebook defines the criteria for all NOAA weather stations

## Imports

In [ ]:
import os
from os import listdir
import pandas as pd
import seaborn as sns


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/My\ Drive/
os. getcwd()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: 'drive/My Drive/'
/content/drive/My Drive


'/content/drive/My Drive'

## Load Data

### Validation of old data source to new.  Old data source had null values filled in as 0

In [ ]:
df_NOAA = pd.read_csv('UMBC_Energy/Use/All_NOAA.csv',sep='|')

In [ ]:
df_NOAA
df_NOAA['state_station_dt'] = df_NOAA['state'] + df_NOAA['station'] + df_NOAA['date']

In [ ]:
df_NOAA

,station_dt_key,state,date,Year,Month,Day,station,PRCP,SNOW,SNWD,AWND,TMAX,TMIN,TDIFF,latitude,longitude,datacoverage,state_station_dt
0,GHCND:USW00014734_2018-01-01,New Jersey,2018-01-01,2018,1,1,GHCND:USW00014734,0.00,0.0,0.0,12.8,20.0,6.0,14.0,40.682500,-74.169400,1.0000,New JerseyGHCND:USW000147342018-01-01
1,GHCND:USW00014734_2018-01-02,New Jersey,2018-01-02,2018,1,2,GHCND:USW00014734,0.00,0.0,0.0,14.3,27.0,12.0,15.0,40.682500,-74.169400,1.0000,New JerseyGHCND:USW000147342018-01-02
2,GHCND:USW00014734_2018-01-03,New Jersey,2018-01-03,2018,1,3,GHCND:USW00014734,0.00,0.0,0.0,5.8,30.0,10.0,20.0,40.682500,-74.169400,1.0000,New JerseyGHCND:USW000147342018-01-03
3,GHCND:USW00014734_2018-01-04,New Jersey,2018-01-04,2018,1,4,GHCND:USW00014734,0.71,8.4,0.0,22.8,29.0,18.0,11.0,40.682500,-74.169400,1.0000,New JerseyGHCND:USW000147342018-01-04
4,GHCND:USW00014734_2018-01-05,New Jersey,2018-01-05,2018,1,5,GHCND:USW00014734,0.00,0.0,7.1,22.6,18.0,9.0,9.0,40.682500,-74.169400,1.0000,New JerseyGHCND:USW000147342018-01-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229136,GHCND:US1DCDC0009_2016-12-16,District of Columbia,2016-12-16,2016,12,16,GHCND:US1DCDC0009,0.00,0.0,0.0,0.0,0.0,0.0,0.0,38.876594,-77.022402,0.4948,District of ColumbiaGHCND:US1DCDC00092016-12-16
229137,GHCND:US1DCDC0009_2016-12-19,District of Columbia,2016-12-19,2016,12,19,GHCND:US1DCDC0009,0.01,0.0,0.0,0.0,0.0,0.0,0.0,38.876594,-77.022402,0.4948,District of ColumbiaGHCND:US1DCDC00092016-12-19
229138,GHCND:US1DCDC0009_2016-12-20,District of Columbia,2016-12-20,2016,12,20,GHCND:US1DCDC0009,0.00,0.0,0.0,0.0,0.0,0.0,0.0,38.876594,-77.022402,0.4948,District of ColumbiaGHCND:US1DCDC00092016-12-20
229139,GHCND:US1DCDC0009_2016-12-22,District of Columbia,2016-12-22,2016,12,22,GHCND:US1DCDC0009,0.00,0.0,0.0,0.0,0.0,0.0,0.0,38.876594,-77.022402,0.4948,District of ColumbiaGHCND:US1DCDC00092016-12-22


In [ ]:
df_NOAA.nunique()

station_dt_key      225170
state                    8
date                  2014
Year                     6
Month                   12
Day                     31
station                141
PRCP                   435
SNOW                   133
SNWD                   104
AWND                   117
TMAX                   112
TMIN                   114
TDIFF                  173
latitude               141
longitude              141
datacoverage            19
state_station_dt    229141
dtype: int64

## New Data Source for Weather Stations

In [ ]:
df_NOAA_Raw_2020 = pd.read_csv('UMBC_Energy/Raw/Raw_2020_NOAA.csv',sep='|')
df_NOAA_Raw_2019 = pd.read_csv('UMBC_Energy/Raw/Raw_2019_NOAA.csv',sep='|')
df_NOAA_Raw_2018 = pd.read_csv('UMBC_Energy/Raw/Raw_2018_NOAA.csv',sep='|')
df_NOAA_Raw_2017 = pd.read_csv('UMBC_Energy/Raw/Raw_2017_NOAA.csv',sep='|')
df_NOAA_Raw_2016 = pd.read_csv('UMBC_Energy/Raw/Raw_2016_NOAA.csv',sep='|')
df_NOAA_Raw_2015 = pd.read_csv('UMBC_Energy/Raw/Raw_2015_NOAA.csv',sep='|')

In [ ]:
df_NOAA_Raw = pd.concat([df_NOAA_Raw_2020,df_NOAA_Raw_2019,df_NOAA_Raw_2018,df_NOAA_Raw_2017,df_NOAA_Raw_2016,df_NOAA_Raw_2015])
df_NOAA_Raw = df_NOAA_Raw.drop_duplicates('station_dt_key')

In [ ]:
df_NOAA_Raw.nunique()

station_dt_key    225170
date                2014
station              141
PRCP                 435
SNOW                 133
SNWD                 104
AWND                 117
TMAX                 112
TMIN                 114
dtype: int64

## Load dataset that has the state of each weather station

In [ ]:
df_station_details = pd.read_csv('UMBC_Energy/Raw/station_details.csv',sep='|')

In [ ]:
df_station_details

,elevation,mindate,maxdate,latitude,name,datacoverage,id,elevationUnit,longitude,state,location_type,location
0,11.9,1865-06-01,1959-06-30,40.233330,"TRENTON NOTTINGHAMWA, NJ US",0.7742,GHCND:USC00288878,METERS,-74.766670,New Jersey,GHCND,USC00288878
1,3.0,1873-12-01,2012-10-03,40.463300,"SANDY HOOK, NJ US",1.0000,GHCND:USC00287865,METERS,-74.005500,New Jersey,GHCND,USC00287865
2,3.0,1873-12-10,2020-07-18,39.377780,"ATLANTIC CITY MARINA, NJ US",1.0000,GHCND:USW00013724,METERS,-74.423610,New Jersey,GHCND,USW00013724
3,30.5,1893-01-01,2020-07-19,40.266390,"HIGHTSTOWN 2 W, NJ US",1.0000,GHCND:USC00283951,METERS,-74.564170,New Jersey,GHCND,USC00283951
4,85.3,1893-01-01,2020-07-19,40.891740,"BOONTON 1 SE, NJ US",1.0000,GHCND:USC00280907,METERS,-74.396350,New Jersey,GHCND,USC00280907
...,...,...,...,...,...,...,...,...,...,...,...,...
728,198.1,1908-11-01,1964-09-30,39.383330,"ST MARYS, WV US",0.9672,GHCND:USC00467875,METERS,-81.200000,West Virginia,GHCND,USC00467875
729,548.0,1909-05-01,1989-06-30,39.666670,"BRANDONVILLE, WV US",0.7245,GHCND:USC00461083,METERS,-79.616670,West Virginia,GHCND,USC00461083
730,1086.6,1912-01-01,1997-12-31,38.610560,"CHEAT BRIDGE, WV US",0.1803,GHCND:USC00461591,METERS,-79.877220,West Virginia,GHCND,USC00461591
731,622.4,1912-04-01,1920-11-30,38.300000,"HOLCOMB, WV US",0.9998,GHCND:USC00464230,METERS,-80.433333,West Virginia,GHCND,USC00464230


In [ ]:
df_station_details.nunique()

elevation        383
mindate          274
maxdate          459
latitude         497
name             630
datacoverage     365
id               715
elevationUnit      1
longitude        569
state              8
location_type      4
location         715
dtype: int64

##### Some ID's appeared in multiple states so had to keep one of them.  There wasn't a value gain or loss by dropping duplicates after the first id was found

In [ ]:
ids = df_station_details["id"]
df_station_details[ids.isin(ids[ids.duplicated()])].sort_values("id").reset_index()

,index,elevation,mindate,maxdate,latitude,name,datacoverage,id,elevationUnit,longitude,state,location_type,location
0,130,275.8,1893-01-01,2010-12-31,39.666670,"EDGEMONT, MD US",1.0000,GHCND:USC00182770,METERS,-77.550000,Pennsylvania,GHCND,USC00182770
1,450,275.8,1893-01-01,2010-12-31,39.666670,"EDGEMONT, MD US",1.0000,GHCND:USC00182770,METERS,-77.550000,Maryland,GHCND,USC00182770
2,684,116.1,1895-01-01,1974-05-31,39.700000,"HANCOCK, MD US",0.5677,GHCND:USC00184025,METERS,-78.183330,West Virginia,GHCND,USC00184025
3,478,116.1,1895-01-01,1974-05-31,39.700000,"HANCOCK, MD US",0.5677,GHCND:USC00184025,METERS,-78.183330,Maryland,GHCND,USC00184025
4,7,18.3,1893-01-01,1949-12-31,40.366670,"LAMBERTVILLE RIVER, PA US",0.9430,GHCND:USC00284653,METERS,-74.950000,New Jersey,GHCND,USC00284653
5,134,18.3,1893-01-01,1949-12-31,40.366670,"LAMBERTVILLE RIVER, PA US",0.9430,GHCND:USC00284653,METERS,-74.950000,Pennsylvania,GHCND,USC00284653
6,509,735.2,1902-05-01,2020-07-19,39.272800,"BAYARD, WV US",1.0000,GHCND:USC00460527,METERS,-79.368400,Maryland,GHCND,USC00460527
7,712,735.2,1902-05-01,2020-07-19,39.272800,"BAYARD, WV US",1.0000,GHCND:USC00460527,METERS,-79.368400,West Virginia,GHCND,USC00460527
8,511,223.1,1904-04-01,1975-08-31,39.616670,"BERKELEY SPRINGS, WV US",0.7661,GHCND:USC00460710,METERS,-78.216670,Maryland,GHCND,USC00460710
9,718,223.1,1904-04-01,1975-08-31,39.616670,"BERKELEY SPRINGS, WV US",0.7661,GHCND:USC00460710,METERS,-78.216670,West Virginia,GHCND,USC00460710


In [ ]:
df_station_details = df_station_details.drop_duplicates('id')

In [ ]:
df_station_details

,elevation,mindate,maxdate,latitude,name,datacoverage,id,elevationUnit,longitude,state,location_type,location
0,11.9,1865-06-01,1959-06-30,40.233330,"TRENTON NOTTINGHAMWA, NJ US",0.7742,GHCND:USC00288878,METERS,-74.766670,New Jersey,GHCND,USC00288878
1,3.0,1873-12-01,2012-10-03,40.463300,"SANDY HOOK, NJ US",1.0000,GHCND:USC00287865,METERS,-74.005500,New Jersey,GHCND,USC00287865
2,3.0,1873-12-10,2020-07-18,39.377780,"ATLANTIC CITY MARINA, NJ US",1.0000,GHCND:USW00013724,METERS,-74.423610,New Jersey,GHCND,USW00013724
3,30.5,1893-01-01,2020-07-19,40.266390,"HIGHTSTOWN 2 W, NJ US",1.0000,GHCND:USC00283951,METERS,-74.564170,New Jersey,GHCND,USC00283951
4,85.3,1893-01-01,2020-07-19,40.891740,"BOONTON 1 SE, NJ US",1.0000,GHCND:USC00280907,METERS,-74.396350,New Jersey,GHCND,USC00280907
...,...,...,...,...,...,...,...,...,...,...,...,...
728,198.1,1908-11-01,1964-09-30,39.383330,"ST MARYS, WV US",0.9672,GHCND:USC00467875,METERS,-81.200000,West Virginia,GHCND,USC00467875
729,548.0,1909-05-01,1989-06-30,39.666670,"BRANDONVILLE, WV US",0.7245,GHCND:USC00461083,METERS,-79.616670,West Virginia,GHCND,USC00461083
730,1086.6,1912-01-01,1997-12-31,38.610560,"CHEAT BRIDGE, WV US",0.1803,GHCND:USC00461591,METERS,-79.877220,West Virginia,GHCND,USC00461591
731,622.4,1912-04-01,1920-11-30,38.300000,"HOLCOMB, WV US",0.9998,GHCND:USC00464230,METERS,-80.433333,West Virginia,GHCND,USC00464230


### Merge to pair station with State

In [ ]:
df_NOAA_Raw.merge(df_station_details,left_on='station',right_on='id',how='left').sort_values('date')

,station_dt_key,date,station,PRCP,SNOW,SNWD,AWND,TMAX,TMIN,elevation,mindate,maxdate,latitude,name,datacoverage,id,elevationUnit,longitude,state,location_type,location
223960,GHCND:US1DEKN0001_2015-01-01,2015-01-01T00:00:00,GHCND:US1DEKN0001,0.0,0.0,NaN,NaN,NaN,NaN,9.8,2009-09-01,2020-06-22,39.03257,"FELTON 3.6 NE, DE US",0.9991,GHCND:US1DEKN0001,METERS,-75.516933,Delaware,GHCND,US1DEKN0001
212223,GHCND:USC00283662_2015-01-01,2015-01-01T00:00:00,GHCND:USC00283662,0.0,0.0,0.0,NaN,32.0,19.0,38.1,1893-01-01,2020-07-19,39.64415,"HAMMONTON 1 NE, NJ US",1.0000,GHCND:USC00283662,METERS,-74.804780,New Jersey,GHCND,USC00283662
195710,GHCND:USC00442635_2015-01-01,2015-01-01T00:00:00,GHCND:USC00442635,0.0,NaN,NaN,NaN,41.0,27.0,3.0,1909-11-01,2015-10-01,37.38760,"EASTVILLE, VA US",1.0000,GHCND:USC00442635,METERS,-75.974400,Virginia,GHCND,USC00442635
197760,GHCND:USC00440327_2015-01-01,2015-01-01T00:00:00,GHCND:USC00440327,0.0,0.0,0.0,NaN,46.0,22.0,67.1,1893-01-01,2020-07-18,37.75000,"ASHLAND, VA US",1.0000,GHCND:USC00440327,METERS,-77.483300,Virginia,GHCND,USC00440327
201773,GHCND:USC00366233_2015-01-01,2015-01-01T00:00:00,GHCND:USC00366233,0.0,0.0,0.0,NaN,30.0,17.0,251.5,1892-05-01,2020-01-31,41.01720,"NEW CASTLE 1 N, PA US",1.0000,GHCND:USC00366233,METERS,-80.361500,Pennsylvania,GHCND,USC00366233
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3207,GHCND:USC00461220_2020-07-06,2020-07-06T00:00:00,GHCND:USC00461220,0.0,0.0,0.0,NaN,95.0,64.0,443.5,1888-01-01,2020-07-19,38.98030,"BUCKHANNON, WV US",1.0000,GHCND:USC00461220,METERS,-80.220100,West Virginia,GHCND,USC00461220
10026,GHCND:USC00338822_2020-07-06,2020-07-06T00:00:00,GHCND:USC00338822,0.0,0.0,0.0,NaN,93.0,65.0,228.6,1893-02-01,2020-07-19,41.51830,"WAUSEON WATER PLANT, OH US",1.0000,GHCND:USC00338822,METERS,-84.145200,Ohio,GHCND,USC00338822
9381,GHCND:USC00361920_2020-07-06,2020-07-06T00:00:00,GHCND:USC00361920,0.0,0.0,0.0,NaN,95.0,67.0,131.1,1840-02-01,2020-07-19,40.22580,"CARLISLE WATER PLANT, PA US",1.0000,GHCND:USC00361920,METERS,-77.189400,Pennsylvania,GHCND,USC00361920
9510,GHCND:USC00361485_2020-07-06,2020-07-06T00:00:00,GHCND:USC00361485,NaN,NaN,0.0,NaN,90.0,57.0,317.0,1884-11-01,2020-07-16,41.19220,"CLARION 3 SW, PA US",1.0000,GHCND:USC00361485,METERS,-79.436100,Pennsylvania,GHCND,USC00361485


In [ ]:
df_NOAA_Raw.merge(df_station_details,left_on='station',right_on='id',how='left').sort_values('date').reset_index()

,index,station_dt_key,date,station,PRCP,SNOW,SNWD,AWND,TMAX,TMIN,elevation,mindate,maxdate,latitude,name,datacoverage,id,elevationUnit,longitude,state,location_type,location
0,223960,GHCND:US1DEKN0001_2015-01-01,2015-01-01T00:00:00,GHCND:US1DEKN0001,0.0,0.0,NaN,NaN,NaN,NaN,9.8,2009-09-01,2020-06-22,39.03257,"FELTON 3.6 NE, DE US",0.9991,GHCND:US1DEKN0001,METERS,-75.516933,Delaware,GHCND,US1DEKN0001
1,212223,GHCND:USC00283662_2015-01-01,2015-01-01T00:00:00,GHCND:USC00283662,0.0,0.0,0.0,NaN,32.0,19.0,38.1,1893-01-01,2020-07-19,39.64415,"HAMMONTON 1 NE, NJ US",1.0000,GHCND:USC00283662,METERS,-74.804780,New Jersey,GHCND,USC00283662
2,195710,GHCND:USC00442635_2015-01-01,2015-01-01T00:00:00,GHCND:USC00442635,0.0,NaN,NaN,NaN,41.0,27.0,3.0,1909-11-01,2015-10-01,37.38760,"EASTVILLE, VA US",1.0000,GHCND:USC00442635,METERS,-75.974400,Virginia,GHCND,USC00442635
3,197760,GHCND:USC00440327_2015-01-01,2015-01-01T00:00:00,GHCND:USC00440327,0.0,0.0,0.0,NaN,46.0,22.0,67.1,1893-01-01,2020-07-18,37.75000,"ASHLAND, VA US",1.0000,GHCND:USC00440327,METERS,-77.483300,Virginia,GHCND,USC00440327
4,201773,GHCND:USC00366233_2015-01-01,2015-01-01T00:00:00,GHCND:USC00366233,0.0,0.0,0.0,NaN,30.0,17.0,251.5,1892-05-01,2020-01-31,41.01720,"NEW CASTLE 1 N, PA US",1.0000,GHCND:USC00366233,METERS,-80.361500,Pennsylvania,GHCND,USC00366233
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225165,3207,GHCND:USC00461220_2020-07-06,2020-07-06T00:00:00,GHCND:USC00461220,0.0,0.0,0.0,NaN,95.0,64.0,443.5,1888-01-01,2020-07-19,38.98030,"BUCKHANNON, WV US",1.0000,GHCND:USC00461220,METERS,-80.220100,West Virginia,GHCND,USC00461220
225166,10026,GHCND:USC00338822_2020-07-06,2020-07-06T00:00:00,GHCND:USC00338822,0.0,0.0,0.0,NaN,93.0,65.0,228.6,1893-02-01,2020-07-19,41.51830,"WAUSEON WATER PLANT, OH US",1.0000,GHCND:USC00338822,METERS,-84.145200,Ohio,GHCND,USC00338822
225167,9381,GHCND:USC00361920_2020-07-06,2020-07-06T00:00:00,GHCND:USC00361920,0.0,0.0,0.0,NaN,95.0,67.0,131.1,1840-02-01,2020-07-19,40.22580,"CARLISLE WATER PLANT, PA US",1.0000,GHCND:USC00361920,METERS,-77.189400,Pennsylvania,GHCND,USC00361920
225168,9510,GHCND:USC00361485_2020-07-06,2020-07-06T00:00:00,GHCND:USC00361485,NaN,NaN,0.0,NaN,90.0,57.0,317.0,1884-11-01,2020-07-16,41.19220,"CLARION 3 SW, PA US",1.0000,GHCND:USC00361485,METERS,-79.436100,Pennsylvania,GHCND,USC00361485


In [ ]:
df_NOAA_Raw_merge = df_NOAA_Raw.merge(df_station_details,left_on='station',right_on='id',how='left').sort_values('date').reset_index().drop(columns=['index'])

## **Load All State and Abbreviations**

In [ ]:
df_state_abbv = pd.read_csv('UMBC_Energy/Universal_Data/state_abb.csv',sep=',')
df_state_abbv = df_state_abbv.drop(columns='Abbrev')
df_state_abbv.head()

,State,Code
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


####Merge two datasets to get state abbreviations 

In [ ]:
df_NOAA_merge_state = df_NOAA_Raw_merge.merge(df_state_abbv,left_on = 'state', right_on = 'State',how='left')

In [ ]:
df_NOAA_merge_state

,station_dt_key,date,station,PRCP,SNOW,SNWD,AWND,TMAX,TMIN,elevation,mindate,maxdate,latitude,name,datacoverage,id,elevationUnit,longitude,state,location_type,location,State,Code
0,GHCND:US1DEKN0001_2015-01-01,2015-01-01T00:00:00,GHCND:US1DEKN0001,0.0,0.0,NaN,NaN,NaN,NaN,9.8,2009-09-01,2020-06-22,39.03257,"FELTON 3.6 NE, DE US",0.9991,GHCND:US1DEKN0001,METERS,-75.516933,Delaware,GHCND,US1DEKN0001,Delaware,DE
1,GHCND:USC00283662_2015-01-01,2015-01-01T00:00:00,GHCND:USC00283662,0.0,0.0,0.0,NaN,32.0,19.0,38.1,1893-01-01,2020-07-19,39.64415,"HAMMONTON 1 NE, NJ US",1.0000,GHCND:USC00283662,METERS,-74.804780,New Jersey,GHCND,USC00283662,New Jersey,NJ
2,GHCND:USC00442635_2015-01-01,2015-01-01T00:00:00,GHCND:USC00442635,0.0,NaN,NaN,NaN,41.0,27.0,3.0,1909-11-01,2015-10-01,37.38760,"EASTVILLE, VA US",1.0000,GHCND:USC00442635,METERS,-75.974400,Virginia,GHCND,USC00442635,Virginia,VA
3,GHCND:USC00440327_2015-01-01,2015-01-01T00:00:00,GHCND:USC00440327,0.0,0.0,0.0,NaN,46.0,22.0,67.1,1893-01-01,2020-07-18,37.75000,"ASHLAND, VA US",1.0000,GHCND:USC00440327,METERS,-77.483300,Virginia,GHCND,USC00440327,Virginia,VA
4,GHCND:USC00366233_2015-01-01,2015-01-01T00:00:00,GHCND:USC00366233,0.0,0.0,0.0,NaN,30.0,17.0,251.5,1892-05-01,2020-01-31,41.01720,"NEW CASTLE 1 N, PA US",1.0000,GHCND:USC00366233,METERS,-80.361500,Pennsylvania,GHCND,USC00366233,Pennsylvania,PA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225165,GHCND:USC00461220_2020-07-06,2020-07-06T00:00:00,GHCND:USC00461220,0.0,0.0,0.0,NaN,95.0,64.0,443.5,1888-01-01,2020-07-19,38.98030,"BUCKHANNON, WV US",1.0000,GHCND:USC00461220,METERS,-80.220100,West Virginia,GHCND,USC00461220,West Virginia,WV
225166,GHCND:USC00338822_2020-07-06,2020-07-06T00:00:00,GHCND:USC00338822,0.0,0.0,0.0,NaN,93.0,65.0,228.6,1893-02-01,2020-07-19,41.51830,"WAUSEON WATER PLANT, OH US",1.0000,GHCND:USC00338822,METERS,-84.145200,Ohio,GHCND,USC00338822,Ohio,OH
225167,GHCND:USC00361920_2020-07-06,2020-07-06T00:00:00,GHCND:USC00361920,0.0,0.0,0.0,NaN,95.0,67.0,131.1,1840-02-01,2020-07-19,40.22580,"CARLISLE WATER PLANT, PA US",1.0000,GHCND:USC00361920,METERS,-77.189400,Pennsylvania,GHCND,USC00361920,Pennsylvania,PA
225168,GHCND:USC00361485_2020-07-06,2020-07-06T00:00:00,GHCND:USC00361485,NaN,NaN,0.0,NaN,90.0,57.0,317.0,1884-11-01,2020-07-16,41.19220,"CLARION 3 SW, PA US",1.0000,GHCND:USC00361485,METERS,-79.436100,Pennsylvania,GHCND,USC00361485,Pennsylvania,PA


In [ ]:
#df_NOAA_merge_state.to_csv('UMBC_Energy/Raw/Raw_All_NOAA.csv',sep='|',index=False)

In [ ]:
df_NOAA_merge_state.groupby('Code').count()

,station_dt_key,date,station,PRCP,SNOW,SNWD,AWND,TMAX,TMIN,elevation,mindate,maxdate,latitude,name,datacoverage,id,elevationUnit,longitude,state,location_type,location,State
Code,,,,,,,,,,,,,,,,,,,,,,
DC,7093,7093,7093,6989,4111,3292,0,3588,3613,7093,7093,7093,7093,7093,7093,7093,7093,7093,7093,7093,7093,7093
DE,40174,40174,40174,39781,23026,7074,4009,7140,7134,40174,40174,40174,40174,40174,40174,40174,40174,40174,40174,40174,40174,40174
MD,11874,11874,11874,11034,8407,8372,2350,11151,11121,11874,11874,11874,11874,11874,11874,11874,11874,11874,11874,11874,11874,11874
NJ,27492,27492,27492,26765,23687,23120,3956,25269,25242,27492,27492,27492,27492,27492,27492,27492,27492,27492,27492,27492,27492,27492
OH,35843,35843,35843,35728,23850,24060,0,29837,29662,35843,35843,35843,35843,35843,35843,35843,35843,35843,35843,35843,35843,35843
PA,31586,31586,31586,31228,28791,28711,0,30375,30371,31586,31586,31586,31586,31586,31586,31586,31586,31586,31586,31586,31586,31586
VA,41915,41915,41915,41379,35201,35177,0,35637,35547,41915,41915,41915,41915,41915,41915,41915,41915,41915,41915,41915,41915,41915
WV,29193,29193,29193,28428,25217,25279,2013,26256,26303,29193,29193,29193,29193,29193,29193,29193,29193,29193,29193,29193,29193,29193


# ** Below: Selection Criteria = Sort on amount of occurences.  Search for stations with all features filled with values similar to top occurence stations.  More features with a value that is similar to the station who appear most frequent is preferred and selected for analysis**

## DC

In [ ]:
#Selected Top 2 selected
df_NOAA_state_group = df_NOAA_merge_state[df_NOAA_merge_state.datacoverage >= .99]

df_NOAA_state_group.query('Code == "DC"').groupby('id').count().sort_values('station',ascending=False).head(6)

,station_dt_key,date,station,PRCP,SNOW,SNWD,AWND,TMAX,TMIN,elevation,mindate,maxdate,latitude,name,datacoverage,elevationUnit,longitude,state,location_type,location,State,Code
id,,,,,,,,,,,,,,,,,,,,,,
GHCND:USC00186350,1876,1876,1876,1861,1492,1495,0,1849,1853,1876,1876,1876,1876,1876,1876,1876,1876,1876,1876,1876,1876,1876
GHCND:USC00182325,1844,1844,1844,1815,1791,1786,0,1739,1760,1844,1844,1844,1844,1844,1844,1844,1844,1844,1844,1844,1844,1844
GHCND:US1DCDC0014,1289,1289,1289,1264,701,0,0,0,0,1289,1289,1289,1289,1289,1289,1289,1289,1289,1289,1289,1289,1289
GHCND:US1DCDC0019,972,972,972,971,61,3,0,0,0,972,972,972,972,972,972,972,972,972,972,972,972,972
GHCND:US1DCDC0010,17,17,17,15,2,1,0,0,0,17,17,17,17,17,17,17,17,17,17,17,17,17


## DE

In [ ]:
#Selected First 2 and Tail id
df_NOAA_state_group = df_NOAA_merge_state[df_NOAA_merge_state.datacoverage >= .99]

df_NOAA_state_group.query('Code == "DE"').groupby('id').count().sort_values('station',ascending=False).head(9)

,station_dt_key,date,station,PRCP,SNOW,SNWD,AWND,TMAX,TMIN,elevation,mindate,maxdate,latitude,name,datacoverage,elevationUnit,longitude,state,location_type,location,State,Code
id,,,,,,,,,,,,,,,,,,,,,,
GHCND:USW00013781,2013,2013,2013,2010,2007,2004,2010,2001,2000,2013,2013,2013,2013,2013,2013,2013,2013,2013,2013,2013,2013,2013
GHCND:USW00013764,2013,2013,2013,2012,2,5,1999,2009,2006,2013,2013,2013,2013,2013,2013,2013,2013,2013,2013,2013,2013,2013
GHCND:US1DENC0010,2003,2003,2003,2003,1205,72,0,0,0,2003,2003,2003,2003,2003,2003,2003,2003,2003,2003,2003,2003,2003
GHCND:US1DESS0035,2000,2000,2000,2000,1199,4,0,0,0,2000,2000,2000,2000,2000,2000,2000,2000,2000,2000,2000,2000,2000
GHCND:US1DENC0002,1998,1998,1998,1996,1557,911,0,0,0,1998,1998,1998,1998,1998,1998,1998,1998,1998,1998,1998,1998,1998
GHCND:US1DESS0003,1987,1987,1987,1975,1725,295,0,0,0,1987,1987,1987,1987,1987,1987,1987,1987,1987,1987,1987,1987,1987
GHCND:US1DESS0025,1956,1956,1956,1942,214,6,0,0,0,1956,1956,1956,1956,1956,1956,1956,1956,1956,1956,1956,1956,1956
GHCND:US1DESS0002,1952,1952,1952,1952,1152,31,0,0,0,1952,1952,1952,1952,1952,1952,1952,1952,1952,1952,1952,1952,1952
GHCND:USC00072730,1886,1886,1886,1814,1713,1705,0,1882,1880,1886,1886,1886,1886,1886,1886,1886,1886,1886,1886,1886,1886,1886


## MD

In [ ]:
#Selected Top 5
df_NOAA_state_group = df_NOAA_merge_state[df_NOAA_merge_state.datacoverage >= .99]

df_NOAA_state_group.query('Code == "MD"').groupby('id').count().sort_values('station',ascending=False).head(6)

,station_dt_key,date,station,PRCP,SNOW,SNWD,AWND,TMAX,TMIN,elevation,mindate,maxdate,latitude,name,datacoverage,elevationUnit,longitude,state,location_type,location,State,Code
id,,,,,,,,,,,,,,,,,,,,,,
GHCND:USC00460527,1995,1995,1995,1994,1988,1978,0,1987,1987,1995,1995,1995,1995,1995,1995,1995,1995,1995,1995,1995,1995,1995
GHCND:USC00186620,1986,1986,1986,1984,1980,1976,0,1981,1974,1986,1986,1986,1986,1986,1986,1986,1986,1986,1986,1986,1986,1986
GHCND:USW00013701,1981,1981,1981,1791,1969,1969,1347,1973,1970,1981,1981,1981,1981,1981,1981,1981,1981,1981,1981,1981,1981,1981
GHCND:USC00188380,1853,1853,1853,1844,249,244,0,1852,1850,1853,1853,1853,1853,1853,1853,1853,1853,1853,1853,1853,1853,1853
GHCND:USC00188405,1316,1316,1316,1011,764,759,0,1148,1152,1316,1316,1316,1316,1316,1316,1316,1316,1316,1316,1316,1316,1316
GHCND:USC00187330,1276,1276,1276,1067,226,224,0,869,848,1276,1276,1276,1276,1276,1276,1276,1276,1276,1276,1276,1276,1276


## NJ

In [ ]:
#Selected Top 5

df_NOAA_state_group = df_NOAA_merge_state[df_NOAA_merge_state.datacoverage >= .99]

df_NOAA_state_group.query('Code == "NJ"').groupby('id').count().sort_values('station',ascending=False).head(6)

,station_dt_key,date,station,PRCP,SNOW,SNWD,AWND,TMAX,TMIN,elevation,mindate,maxdate,latitude,name,datacoverage,elevationUnit,longitude,state,location_type,location,State,Code
id,,,,,,,,,,,,,,,,,,,,,,
GHCND:USC00288644,2014,2014,2014,1913,1979,1972,0,1996,1989,2014,2014,2014,2014,2014,2014,2014,2014,2014,2014,2014,2014,2014
GHCND:USC00283662,2013,2013,2013,1928,1991,1991,0,2005,2003,2013,2013,2013,2013,2013,2013,2013,2013,2013,2013,2013,2013,2013
GHCND:USW00014734,2013,2013,2013,2011,2009,2007,2012,2004,2001,2013,2013,2013,2013,2013,2013,2013,2013,2013,2013,2013,2013,2013
GHCND:USW00013724,2011,2011,2011,2004,0,0,1944,1998,1998,2011,2011,2011,2011,2011,2011,2011,2011,2011,2011,2011,2011,2011
GHCND:USC00283029,2010,2010,2010,1998,1798,1807,0,1993,1986,2010,2010,2010,2010,2010,2010,2010,2010,2010,2010,2010,2010,2010
GHCND:USC00281582,2009,2009,2009,1653,1721,1461,0,1997,1996,2009,2009,2009,2009,2009,2009,2009,2009,2009,2009,2009,2009,2009


## OH

In [ ]:
#Selected Top 5
df_NOAA_state_group = df_NOAA_merge_state[df_NOAA_merge_state.datacoverage >= .99]

df_NOAA_state_group.query('Code == "OH"').groupby('id').count().sort_values('station',ascending=False).head(6)

,station_dt_key,date,station,PRCP,SNOW,SNWD,AWND,TMAX,TMIN,elevation,mindate,maxdate,latitude,name,datacoverage,elevationUnit,longitude,state,location_type,location,State,Code
id,,,,,,,,,,,,,,,,,,,,,,
GHCND:USC00330862,2011,2011,2011,2011,1359,1346,0,2005,2004,2011,2011,2011,2011,2011,2011,2011,2011,2011,2011,2011,2011,2011
GHCND:USC00338822,2011,2011,2011,2010,2008,2003,0,2002,2001,2011,2011,2011,2011,2011,2011,2011,2011,2011,2011,2011,2011,2011
GHCND:USC00338609,2007,2007,2007,2006,1944,1942,0,2002,1992,2007,2007,2007,2007,2007,2007,2007,2007,2007,2007,2007,2007,2007
GHCND:USC00336196,2005,2005,2005,2005,1976,1974,0,3,2,2005,2005,2005,2005,2005,2005,2005,2005,2005,2005,2005,2005,2005
GHCND:USC00338769,2003,2003,2003,1998,1580,1578,0,1980,1971,2003,2003,2003,2003,2003,2003,2003,2003,2003,2003,2003,2003,2003
GHCND:USC00338830,1984,1984,1984,1974,1791,1786,0,1961,1956,1984,1984,1984,1984,1984,1984,1984,1984,1984,1984,1984,1984,1984


## PA

In [ ]:
# Selected 
df_NOAA_state_group = df_NOAA_merge_state[df_NOAA_merge_state.datacoverage >= .99]

df_NOAA_state_group.query('Code == "PA"').groupby('id').count().sort_values('station',ascending=False).head(6)

,station_dt_key,date,station,PRCP,SNOW,SNWD,AWND,TMAX,TMIN,elevation,mindate,maxdate,latitude,name,datacoverage,elevationUnit,longitude,state,location_type,location,State,Code
id,,,,,,,,,,,,,,,,,,,,,,
GHCND:USC00367409,2014,2014,2014,2014,2012,2010,0,2009,2009,2014,2014,2014,2014,2014,2014,2014,2014,2014,2014,2014,2014,2014
GHCND:USC00363632,2014,2014,2014,2007,1942,1938,0,2007,2006,2014,2014,2014,2014,2014,2014,2014,2014,2014,2014,2014,2014,2014
GHCND:USC00367477,2014,2014,2014,2014,1981,1977,0,2009,2008,2014,2014,2014,2014,2014,2014,2014,2014,2014,2014,2014,2014,2014
GHCND:USC00367931,2014,2014,2014,1939,2001,1993,0,1886,1894,2014,2014,2014,2014,2014,2014,2014,2014,2014,2014,2014,2014,2014
GHCND:USC00368449,2014,2014,2014,2014,2008,2007,0,2006,2004,2014,2014,2014,2014,2014,2014,2014,2014,2014,2014,2014,2014,2014
GHCND:USC00361920,2011,2011,2011,2011,2007,2007,0,2006,2005,2011,2011,2011,2011,2011,2011,2011,2011,2011,2011,2011,2011,2011


## VA

In [ ]:
df_NOAA_state_group = df_NOAA_merge_state[df_NOAA_merge_state.datacoverage >= .99]

df_NOAA_state_group.query('Code == "VA"').groupby('id').count().sort_values('station',ascending=False).head(6)

,station_dt_key,date,station,PRCP,SNOW,SNWD,AWND,TMAX,TMIN,elevation,mindate,maxdate,latitude,name,datacoverage,elevationUnit,longitude,state,location_type,location,State,Code
id,,,,,,,,,,,,,,,,,,,,,,
GHCND:USC00443272,2014,2014,2014,2013,2004,2002,0,2003,2003,2014,2014,2014,2014,2014,2014,2014,2014,2014,2014,2014,2014,2014
GHCND:USC00449301,2014,2014,2014,2014,2010,2011,0,2005,2004,2014,2014,2014,2014,2014,2014,2014,2014,2014,2014,2014,2014,2014
GHCND:USC00444876,2013,2013,2013,2013,2013,2011,0,2000,1997,2013,2013,2013,2013,2013,2013,2013,2013,2013,2013,2013,2013,2013
GHCND:USC00441593,2013,2013,2013,2013,2013,2011,0,2008,2008,2013,2013,2013,2013,2013,2013,2013,2013,2013,2013,2013,2013,2013
GHCND:USC00447506,2013,2013,2013,2012,2007,2007,0,2000,1991,2013,2013,2013,2013,2013,2013,2013,2013,2013,2013,2013,2013,2013
GHCND:USC00448062,2011,2011,2011,2010,2008,2005,0,2002,2002,2011,2011,2011,2011,2011,2011,2011,2011,2011,2011,2011,2011,2011


##WV

In [ ]:
df_NOAA_state_group = df_NOAA_merge_state[df_NOAA_merge_state.datacoverage >= .99]

df_NOAA_state_group.query('Code == "WV"').groupby('id').count().sort_values('station',ascending=False).head(6)

,station_dt_key,date,station,PRCP,SNOW,SNWD,AWND,TMAX,TMIN,elevation,mindate,maxdate,latitude,name,datacoverage,elevationUnit,longitude,state,location_type,location,State,Code
id,,,,,,,,,,,,,,,,,,,,,,
GHCND:USW00013736,2013,2013,2013,2010,288,470,2013,2007,2005,2013,2013,2013,2013,2013,2013,2013,2013,2013,2013,2013,2013,2013
GHCND:USC00465224,2012,2012,2012,2010,2010,2009,0,1976,1976,2012,2012,2012,2012,2012,2012,2012,2012,2012,2012,2012,2012,2012
GHCND:USC00461220,2012,2012,2012,1994,1963,1957,0,1936,1957,2012,2012,2012,2012,2012,2012,2012,2012,2012,2012,2012,2012,2012
GHCND:USC00460580,2012,2012,2012,1997,2006,2005,0,1954,1955,2012,2012,2012,2012,2012,2012,2012,2012,2012,2012,2012,2012,2012
GHCND:USC00465353,2011,2011,2011,1998,2009,2006,0,1961,1987,2011,2011,2011,2011,2011,2011,2011,2011,2011,2011,2011,2011,2011
GHCND:USC00466867,2008,2008,2008,2007,2000,1996,0,2002,2000,2008,2008,2008,2008,2008,2008,2008,2008,2008,2008,2008,2008,2008
